In [206]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
dummy_data = pd.read_csv('/Users/henry/Desktop/dummdata - Sheet1.csv')

#Buckets + outlier detection and remove/replacement

#Class for bucketing Data
#Class for outlier detection/replcaement or removal
#Class for one hot encoding
#Class for scaling numerical values

In [207]:
#Even distinact between min and max of each bucket. 
class Bucketing:
    
    def __init__(self, raw_data):
        self.raw_data = raw_data
        if isinstance(raw_data, pd.DataFrame):
            pass
        else:
            raise ValueError('Must pass in a dataframe')
    def even_compartments(self, columns=[], Number_of_buckets = 10, bucket_array =[], replace = False, append=True,
                 min_bucket_size = None, max_bucket_size = None, bottom_bucket_inclusive =True):
        
        data = self.raw_data
        data_T = even_size_buckets(raw_data = data, columns=columns, Number_of_buckets=Number_of_buckets
                                   ,replace=replace,append=append, 
                                   min_bucket_size=min_bucket_size,max_bucket_size=max_bucket_size,
                                   bottom_bucket_inclusive=bottom_bucket_inclusive)
        return data_T
    
    def customized_buckets(self, columns=[], bucket_array=[], replace=False, append=True,bottom_bucket_inclusive=True):
        data = self.raw_data
        data_T = custom_buckets(raw_data = data, columns=columns, 
                                     bucket_array=bucket_array, replace=replace, 
                                     append=append, 
                                     bottom_bucket_inclusive=bottom_bucket_inclusive)
        return data_T
    
    def even_entries_per_bucket(self,columns=[], Number_of_buckets=10, min_bucket_size = None, 
                                max_bucket_size = None, replace = False, append = True, 
                                bottom_bucket_inclusive = True, display_entries = True):
        data = self.raw_data

        data_T = even_entry_buckets(raw_data = data, columns=columns, Number_of_buckets=Number_of_buckets, 
                                    min_bucket_size=min_bucket_size, max_bucket_size=max_bucket_size, 
                                    replace=replace, append=append,
                          bottom_bucket_inclusive=bottom_bucket_inclusive, display_entries=display_entries)
        

        return data_T
        
        
   
    
    
    
            
        
    
        
        
        

In [216]:
#Tests
#Even bucket sizes
df_T_obj = Bucketing(dummy_data)
#df_T_obj.even_compartments(columns=['temperature'],Number_of_buckets=70)
#custom_buckets
#df_T_obj.customized_buckets(columns=['temperature'], bucket_array=[-60],bottom_bucket_inclusive=True)
df_T_obj.even_entries_per_bucket(columns=['temperature','windspeed'], min_bucket_size=180)



8


IndexError: list index out of range

In [215]:
def even_entry_buckets(raw_data,columns, Number_of_buckets, min_bucket_size, max_bucket_size, replace, append, bottom_bucket_inclusive,display_entries):
        bucket_df = pd.DataFrame()
        run_through = 0
        Number_of_buckets = int(Number_of_buckets)
        for column in columns:
            Num_bucks = Number_of_buckets
            Num_bucks -= 1
            entries = list(dummy_data[column])
            if str(min_bucket_size).isnumeric() ==True:
                entries = [number for number in entries if number >= min_bucket_size]
                Num_bucks-=1
            if str(max_bucket_size).isnumeric() == True:
                entries = [number for number in entries if number <= max_bucket_size]
                Num_bucks -=1
            entries.sort()
            items_per_bucket = int(len(entries)/Num_bucks)
            bucket_iter = 0
            buckets = []
            print(Num_bucks)
            for i in range(0, Num_bucks):
                bucket_iter+=int(items_per_bucket-1)
                bucket = entries[bucket_iter]
                buckets.append(bucket)
            if str(min_bucket_size).isnumeric() ==True:
                buckets.append(min_bucket_size)

            if str(max_bucket_size).isnumeric() == True:
                buckets.append(max_bucket_size)
            buckets = list(set(buckets))
            buckets.sort()
            
            just_the_data = Bucketing(raw_data=raw_data).customized_buckets(columns = [column], bucket_array=buckets, replace=replace, append=False, bottom_bucket_inclusive=bottom_bucket_inclusive)
            if run_through == 0:
                bucket_df = just_the_data
            else:
                bucket_df = bucket_df.join(just_the_data)
            run_through +=1
        
        if display_entries == True:
            for column in bucket_df.columns:
                print(column, bucket_df[column].sum())

        base_cols = [column for column in raw_data.columns if column not in columns]
        if append == True:
            base_df = raw_data[base_cols]
            bucket_df = base_df.join(bucket_df)
            
        
        return bucket_df



    


In [210]:
#Custom Buckets
def custom_buckets(raw_data, columns,bucket_array, replace, append, bottom_bucket_inclusive):
        bucket_array.sort()
        try:
            columns[0]
            bucket_array[0]
        except:
            raise ValueError('columns must be a list of columns and bucket_array must be an array of bottom bucket sizes')
        if replace == True or replace == False:
            pass
        else:
            raise ValueError('replace must be boolean')
        if append == True or append == False:
            pass
        else:
            raise ValueError('append must be boolean')
        
        print('custom bucket sizes: ', columns)
        return_frame = pd.DataFrame()
        if bottom_bucket_inclusive == True: 
            for item in columns:
                try: 
                    bucketed_data = pd.DataFrame(raw_data[item], columns=[item])
                except:
                    raise ValueError('no such column: ', item)
                
                try:
                   
                    print('\ncolumn: ', item, 'columns:', str(bucket_array))
                    test = float(bucketed_data[item].iloc[0])
                except:
                    raise ValueError('Bucket column ', item, ' must be numeric')
                    
                for bucket in range(0,len(bucket_array)+1):
                    try:
                        if bucket != 0 and bucket != (len(bucket_array)):
                            bucket_start = bucket_array[bucket-1]
                            bucket_end = bucket_array[bucket]
                            new_column = item +'_>='+ str(bucket_start)+ '_<' + str(bucket_end)
                            bucketed_data[new_column] = bucketed_data.apply(lambda x: 1 if x[item] < bucket_end and x[item] >= bucket_start else 0,axis=1)
                        elif bucket == 0 and bucket != len(bucket_array) + 1:
                            bucket_end = bucket_array[bucket]
                            new_column = item + '_<' + str(bucket_end)
                            bucketed_data[new_column] = bucketed_data.apply(lambda x: 1 if x[item] < bucket_end else 0,axis=1)
                        else:
                            bucket_start = bucket_array[bucket-1]
                            new_column = item + '_>=' + str(bucket_start)
                            bucketed_data[new_column] = bucketed_data.apply(lambda x: 1 if x[item] >= bucket_start else 0,axis=1)
                    except:
                        raise ValueError('Buckets must be an array of bottom bucket values')
                return_frame = pd.concat([return_frame, bucketed_data], axis=1)
                
                
            if append == True and replace==True:
                return_frame = pd.concat([raw_data, return_frame],axis=1)
                return_frame = return_frame.drop(columns=columns)
                return return_frame
            elif append==True and replace==False:
                return_frame = return_frame.drop(columns=columns)
                return_frame = pd.concat([raw_data, return_frame],axis=1)
                return return_frame
            elif append==False and replace==True:
                return_frame = return_frame.drop(columns=columns)
                return return_frame
            elif append==False and replace==False:
                return return_frame
       
        else: 

            for item in columns:
                try: 
                    bucketed_data = pd.DataFrame(raw_data[item], columns=[item])
                except:
                    raise ValueError('no such column: ', item)

                try:

                    print('\ncolumn: ', item, 'columns:', str(bucket_array))
                    test = float(bucketed_data[item].iloc[0])
                except:
                    raise ValueError('Bucket column ', item, ' must be numeric')

                for bucket in range(0,len(bucket_array)+1):
                    try:
                        if bucket != 0 and bucket != (len(bucket_array)):
                            bucket_start = bucket_array[bucket-1]
                            bucket_end = bucket_array[bucket]
                            new_column = item +'_>'+ str(bucket_start)+ '_<=' + str(bucket_end)
                            bucketed_data[new_column] = bucketed_data.apply(lambda x: 1 if x[item] <= bucket_end and x[item] > bucket_start else 0,axis=1)
                        elif bucket == 0 and bucket != len(bucket_array) + 1:
                            bucket_end = bucket_array[bucket]
                            new_column = item + '_<=' + str(bucket_end)
                            bucketed_data[new_column] = bucketed_data.apply(lambda x: 1 if x[item] <= bucket_end else 0,axis=1)
                        else:
                            bucket_start = bucket_array[bucket-1]
                            new_column = item + '_>' + str(bucket_start)
                            bucketed_data[new_column] = bucketed_data.apply(lambda x: 1 if x[item] > bucket_start else 0,axis=1)
                    except:
                        raise ValueError('Buckets must be an array of bottom bucket values')
                return_frame = pd.concat([return_frame, bucketed_data], axis=1)


        if append == True and replace==True:
            return_frame = pd.concat([raw_data, return_frame],axis=1)
            return_frame = return_frame.drop(columns=columns)
            return return_frame
        elif append==True and replace==False:
            return_frame = return_frame.drop(columns=columns)
            return_frame = pd.concat([raw_data, return_frame],axis=1)
            return return_frame
        elif append==False and replace==True:
            return_frame = return_frame.drop(columns=columns)
            return return_frame
        elif append==False and replace==False:
            return return_frame

In [211]:
 #Gives even bucket sizes
def even_size_buckets(raw_data, columns, Number_of_buckets,replace,append, min_bucket_size,max_bucket_size, bottom_bucket_inclusive):
    if columns[0]:
        pass
    else:
        raise ValueError('columns must be a list of columns')
    if str(Number_of_buckets).isnumeric():
        pass
    else:
        raise ValueError('Number of buckets must be a number')
    if replace == True or replace == False:
        pass
    else:
        raise ValueError('replace must be boolean')
    if append == True or append == False:
        pass
    else:
        raise ValueError('append must be boolean')

    print('even bucket sizes: ', columns)
    return_frame = pd.DataFrame()
    if bottom_bucket_inclusive == True: 
        for item in columns:
            try: 
                bucketed_data = pd.DataFrame(raw_data[item], columns=[item])
                data = bucketed_data.values
            except:
                raise ValueError('no such column: ', item)
            if str(min_bucket_size).isnumeric():
                minimum = min_bucket_size
            else:
                minimum = min(data)
            if str(max_bucket_size).isnumeric():
                maximum = max_bucket_size
            else:
                maximum = max(data)
            try:
                bucket_size = (maximum-minimum)/Number_of_buckets
                print('\ncolumn: ', item, '\nmin:', str(minimum), ' max:', str(maximum), ' bucket size:', str(bucket_size))
            except:
                raise ValueError('Bucket column ', item, ' must be numeric')

            bucket_start = float(minimum)


            for bucket in range(0, int(Number_of_buckets)):
                bucket_end = bucket_start + bucket_size
                if bucket == 0:
                    less_than_min_column_name = str(item) + '_<_' + str(minimum)
                    bucketed_data[less_than_min_column_name] = bucketed_data.apply(lambda x: 1 if x[item] < minimum else 0,axis=1)


                column_name = str(item)+ '_>=_' + str(bucket_start) + '_<' + str(bucket_end) 
                bucketed_data[column_name] = bucketed_data.apply(lambda x: 1 if x[item]>= bucket_start and x[item] < bucket_end else 0,axis=1)
                if bucket == int(Number_of_buckets) -1:
                    more_than_max_column_name = str(item) + '_>=_' + str(maximum)
                    bucketed_data[more_than_max_column_name] = bucketed_data.apply(lambda x: 1 if x[item]>= maximum else 0,axis=1)

                bucket_start +=bucket_size
            return_frame = pd.concat([return_frame, bucketed_data], axis=1)
    else:
        for item in columns:
            try: 
                bucketed_data = pd.DataFrame(raw_data[item], columns=[item])
            except:
                raise ValueError('no such column: ', item)
            if str(min_bucket_size).isnumeric():
                minimum = min_bucket_size
            else:
                minimum = min(data)
            if str(max_bucket_size).isnumeric():
                maximum = max_bucket_size
            else:
                maximum = max(data)
            try:
                bucket_size = (maximum-minimum)/Number_of_buckets
                print('\ncolumn: ', item, '\nmin:', str(minimum), ' max:', str(maximum), ' bucket size:', str(bucket_size))
            except:
                raise ValueError('Bucket column ', item, ' must be numeric')

            bucket_start = minimum

            for bucket in range(0, int(Number_of_buckets)):
                bucket_end = bucket_start + bucket_size
                if bucket == 0:
                    less_than_min_column_name = str(item) + '_<=_' + str(minimum)
                    bucketed_data[less_than_min_column_name] = bucketed_data.apply(lambda x: 1 if x[item] <= minimum else 0,axis=1)


                column_name = str(item)+ '_>_' + str(bucket_start) + '_<=' + str(bucket_end) 
                bucketed_data[column_name] = bucketed_data.apply(lambda x: 1 if x[item] > bucket_start and x[item] <= bucket_end else 0,axis=1)
                if bucket == int(Number_of_buckets) -1:
                    more_than_max_column_name = str(item) + '_>_' + str(maximum)
                    bucketed_data[more_than_max_column_name] = bucketed_data.apply(lambda x: 1 if x[item]> maximum else 0,axis=1)

                bucket_start +=bucket_size
            return_frame = pd.concat([return_frame, bucketed_data], axis=1)

    if append == True and replace==True:
        return_frame = pd.concat([raw_data, return_frame],axis=1)
        return_frame = return_frame.drop(columns=columns)
        return return_frame
    elif append==True and replace==False:
        return_frame = return_frame.drop(columns=columns)
        return_frame = pd.concat([raw_data, return_frame],axis=1)
        return return_frame
    elif append==False and replace==True:
        return_frame = return_frame.drop(columns=columns)
        return return_frame
    elif append==False and replace==False:
        return return_frame

    
    